# **Milestone 2: Data Cleaning and Preprocessing**

## **Requirements and Solutions**

### **1. Handle Missing Values**
- **Numerical Features**: Imputed missing values with the median.
- **Categorical Features**: Filled missing values with `'Unknown'`.
- **Specific Columns**:
  - `Description`: Filled with `'No Description'`.
  - `Poster URL`: Filled with `'No URL'`.
  - `Cast`: Filled with `'No Cast Information'`.

### **2. Normalize Data**
- Applied **MinMaxScaler** to scale `IMDb Rating`, `Number of Votes`, and `Release Year` to a range of [0, 1].

### **3. Convert Categorical Features**
- Used **MultiLabelBinarizer** to one-hot encode `Genres`, creating binary columns for each genre.

### **4. Remove Duplicate Entries**
- Combined `Title` and `Release Year` into `Title_Release_Year` for unique identification.
- Removed duplicates based on `Title_Release_Year`.

---

## **Outcome**
The cleaned and preprocessed dataset meets the requirements for missing values, normalization, encoding, and duplicate removal, ensuring consistency and readiness for analysis.


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer

# Load dataset
file_path = '/Users/i2gunshaker/top20k_movies_project2.csv'
movies_df = pd.read_csv(file_path)

# Impute missing 'Release Year' before combining
if movies_df['Release Year'].isnull().sum() > 0:
    median_release_year = movies_df['Release Year'].median()
    movies_df['Release Year'] = movies_df['Release Year'].fillna(median_release_year)

# Combine Title and Release Year
movies_df['Title_Release_Year'] = (
    movies_df['Title'].fillna('Unknown Title') + 
    " (" + movies_df['Release Year'].astype(int).astype(str) + ")"
)

# Remove duplicates
initial_shape = movies_df.shape
movies_df = movies_df.drop_duplicates(subset=['Title_Release_Year'])
final_shape = movies_df.shape
print(f"Removed {initial_shape[0] - final_shape[0]} duplicate rows.")

# Handle missing values
numerical_features = ['IMDb Rating', 'Number of Votes', 'Release Year']
categorical_features = ['Genres']

# Impute numerical features with median
for feature in numerical_features:
    if movies_df[feature].isnull().sum() > 0:
        median = movies_df[feature].median()
        movies_df[feature] = movies_df[feature].fillna(median)

# Impute categorical features with 'Unknown'
for feature in categorical_features:
    if movies_df[feature].isnull().sum() > 0:
        movies_df[feature] = movies_df[feature].fillna('Unknown')

# Handle missing values in specific object columns
if 'Poster URL' in movies_df.columns:
    movies_df['Poster URL'] = movies_df['Poster URL'].fillna('No URL')

if 'Description' in movies_df.columns:
    movies_df['Description'] = movies_df['Description'].fillna('No Description')

if 'Cast' in movies_df.columns:
    movies_df['Cast'] = movies_df['Cast'].fillna('No Cast Information')

# Normalize numerical features
scaler = MinMaxScaler()
normalized_cols = [f"normalized_{feat}" for feat in numerical_features]
movies_df[normalized_cols] = scaler.fit_transform(movies_df[numerical_features])

# One-hot encode 'Genres'
movies_df['Genres'] = movies_df['Genres'].str.split(', ')
mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(
    mlb.fit_transform(movies_df['Genres']),
    columns=[f"Genre_{genre}" for genre in mlb.classes_],
    index=movies_df.index
)
movies_df = pd.concat([movies_df.drop('Genres', axis=1), genres_encoded], axis=1)

# Drop 'Genre_Unknown' if exists
if 'Genre_Unknown' in movies_df.columns:
    movies_df = movies_df.drop('Genre_Unknown', axis=1)

# Select relevant columns
optional_cols = ['Description', 'Poster URL', 'Cast', 'Directors']
existing_optional = [col for col in optional_cols if col in movies_df.columns]

output_columns = [
    'Title_Release_Year', 'Release Year', 'IMDb Rating', 'Number of Votes', 
    'normalized_Release Year', 'normalized_IMDb Rating', 'normalized_Number of Votes'
] + existing_optional + [col for col in movies_df.columns if col.startswith('Genre_')]

movies_df = movies_df[output_columns]

# Save cleaned data
output_file_path = '/Users/i2gunshaker/top11k_movies_cleaned2.csv'
movies_df.to_csv(output_file_path, index=False)
print(movies_df.info())


Removed 0 duplicate rows.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11976 entries, 0 to 11975
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Title_Release_Year          11976 non-null  object 
 1   Release Year                11976 non-null  int64  
 2   IMDb Rating                 11976 non-null  float64
 3   Number of Votes             11976 non-null  int64  
 4   normalized_Release Year     11976 non-null  float64
 5   normalized_IMDb Rating      11976 non-null  float64
 6   normalized_Number of Votes  11976 non-null  float64
 7   Description                 11976 non-null  object 
 8   Poster URL                  11976 non-null  object 
 9   Cast                        11976 non-null  object 
 10  Genre_Action                11976 non-null  int64  
 11  Genre_Adventure             11976 non-null  int64  
 12  Genre_Animation             11976 non-null  int64  
 13  Genre

In [8]:
movies_df

,Title_Release_Year,Release Year,IMDb Rating,Number of Votes,normalized_Release Year,normalized_IMDb Rating,normalized_Number of Votes,Description,Poster URL,Cast,...,Genre_History,Genre_Horror,Genre_Music,Genre_Mystery,Genre_Romance,Genre_Science Fiction,Genre_TV Movie,Genre_Thriller,Genre_War,Genre_Western
0,The Way to the Heart (2022),2022,9.952,124,0.977528,1.000000,0.001204,"Ava, an award-winning chef at a big-city resta...",https://image.tmdb.org/t/p/w500/xFKzEhOhkPwMxW...,"Jon Cor, Meghan Heffern, Peter Graham",...,0,0,0,0,0,0,0,0,0,0
1,BTS: Permission to Dance on Stage - LA (2022),2022,8.866,157,0.977528,0.861691,0.002107,"Purple colors the city of Los Angeles, as BTS ...",https://image.tmdb.org/t/p/w500/9pNGRpOOQpZtqM...,"RM, Jin, SUGA",...,0,0,1,0,0,0,0,0,0,0
2,Rubius X (2022),2022,8.600,141,0.977528,0.827815,0.001669,On the 10th anniversary of the publication of ...,https://image.tmdb.org/t/p/w500/k6KAAVebtdVb0F...,"Rubén Doblas Gundersen, Miguel Ángel Rogel, Al...",...,0,0,0,0,0,0,0,0,0,0
3,Franco Escamilla: Eavesdropping (2022),2022,8.500,85,0.977528,0.815079,0.000137,Franco Escamilla takes the stage in California...,https://image.tmdb.org/t/p/w500/cLaTeAG7kq4x39...,Franco Escamilla,...,0,0,0,0,0,0,0,0,0,0
4,Teasing Master Takagi-san: The Movie (2022),2022,8.500,126,0.977528,0.815079,0.001259,Takagi and Nishikata are in their final year o...,https://image.tmdb.org/t/p/w500/kGyya4tD6mDCGj...,"Yuki Kaji, Rie Takahashi, Inori Minase",...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11971,Triumph of the Will (1935),1935,6.955,268,0.000000,0.618314,0.005144,A showcase of German chancellor and Nazi Party...,https://image.tmdb.org/t/p/w500/hyXCugoQzJ85md...,"Adolf Hitler, Max Amann, Hermann Göring",...,0,0,0,0,0,0,0,0,0,0
11972,Mad Love (1935),1935,6.900,110,0.000000,0.611309,0.000821,An insane surgeon's obsession with an actress ...,https://image.tmdb.org/t/p/w500/lDD8pKd6jSrFLV...,"Peter Lorre, Frances Drake, Colin Clive",...,0,1,0,0,1,0,0,0,0,0
11973,The Raven (1935),1935,6.500,138,0.000000,0.560367,0.001587,A brilliant but deranged neurosurgeon becomes ...,https://image.tmdb.org/t/p/w500/iVMNfBvKe4XXxi...,"Boris Karloff, Bela Lugosi, Lester Matthews",...,0,1,0,0,0,0,0,0,0,0
11974,Mark of the Vampire (1935),1935,6.100,129,0.000000,0.509424,0.001341,"Sir Borotyn, a prominent Prague resident, is d...",https://image.tmdb.org/t/p/w500/A6JN2kIEYwXEHH...,"Lionel Barrymore, Elizabeth Allan, Bela Lugosi",...,0,1,0,1,0,0,0,0,0,0
